[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/cli/geomopt.ipynb)

# Geometry Optimization

## Set up environment (optional)

These steps are required to run this tutorial with Google Colab. To do so, uncomment and run the cell below.

This will replace pre-installed versions of `numpy` and `torch` in Colab with versions that are known to be compatible with `janus-core`.

It may be possible to skip the steps that uninstall and reinstall `torch`, which will save a considerable amount of time.

These instructions but may work for other systems too, but it is typically preferable to prepare a virtual environment separately before running this notebook if possible.

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall numpy -y # Uninstall pre-installed numpy

# ! pip uninstall torch torchaudio torchvision transformers -y # Uninstall pre-installed torch
# ! uv pip install torch==2.5.1 # Install pinned version of torch

# ! uv pip install janus-core[mace,orb,visualise] data-tutorials --system # Install janus-core with MACE, Orb, and WeasWidget, and data-tutorials

# get_ipython().kernel.do_shutdown(restart=True) # Restart kernel to update libraries. This may warn that your session has crashed.

To ensure you have the latest version of `janus-core` installed, compare the output of the following cell to the latest version available at https://pypi.org/project/janus-core/

In [ ]:
from janus_core import __version__

print(__version__)

## Command-line help and options

As with `janus singlepoint`, we can check the options for geometry optimisation:

In [ ]:
! janus geomopt --help

## Running geometry optimisation calculation

First, we'll build a structure to optimise, as we did for single point calculations, but add in a deformation:

In [ ]:
from pathlib import Path

from ase.build import bulk
from ase.io import write
from weas_widget import WeasWidget

Path("../data").mkdir(exist_ok=True)

NaCl = bulk("NaCl", "rocksalt", a=5.63, cubic=True)
NaCl[0].position = [1.5, 1.5, 1.5]

write("../data/NaCl-deformed.xyz", NaCl)

v=WeasWidget()
v.from_ase(NaCl)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

Now we can optimise the geometry of this structure in a similar manner to running `janus singlepoint`:

In [ ]:
! janus geomopt --arch mace_mp --struct ../data/NaCl-deformed.xyz --no-tracker

We can also change the optimisation function used, and specify an even lower force convergence criteria, `fmax` (the maximum force on all individual atoms):

<div class="alert alert-block alert-info">
<b>Tip:</b> The optimizer must be a class defined in ASE.
</div>


In [ ]:
! janus geomopt --arch mace_mp --struct ../data/NaCl-deformed.xyz --optimizer FIRE --fmax 0.005 --no-tracker

As with single point calculations, this saves a results file, corresponding to the optimised structure, as well as a summary and log:

In [ ]:
! ls janus_results/NaCl-deformed*

We can now see the optimised structure:

In [ ]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-opt.extxyz")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

## Trajectories

We can also save the trajectory during optimisation:

In [ ]:
! janus geomopt --arch mace_mp --struct ../data/NaCl-deformed.xyz --write-traj --no-tracker

This creates an additional file, `janus_results/NaCl-deformed-traj.extxyz`:

In [ ]:
! ls janus_results/NaCl-deformed*

This allows us to visualise the optimisation:

In [ ]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

## Cell optimisation

We can also choose to modify the cell vectors during the optimisation. To allow only the cell lengths to change, we can run:

In [ ]:
! janus geomopt --arch mace_mp --struct ../data/NaCl-deformed.xyz --write-traj --opt-cell-lengths --no-tracker

As before, we can visualise this trajectory:

In [ ]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

We can also allow the cell angles to change:

In [ ]:
! janus geomopt --arch mace_mp --struct ../data/NaCl-deformed.xyz --write-traj --opt-cell-fully --no-tracker

Visualising this:

In [ ]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

## Setting the unit cell filter

Cell optimisation is carried out by appling an ASE `filter` to the structure. By default, this is the `FrechetCellFilter`, but you may wish to apply others, such as the `ExpCellFilter`. This is passed as a string, and must correspond to a class defined in ASE.

Key word arguments can also be passed to these filters. For example, we can maintain constant volume using the following configuration file:

In [ ]:
%%writefile geomopt_config_1.yml

arch: mace_mp
struct: ../data/NaCl-deformed.xyz
opt_cell_fully: True
filter: ExpCellFilter
minimize_kwargs:
  filter_kwargs:
    constant_volume: True
tracker: False

<div class="alert alert-block alert-info">
<b>Tip:</b> This is equivalent to:

--struct ../data/NaCl-deformed.xyz --opt-cell-fully --filter ExpCellFilter --minimize-kwargs "{'filter_kwargs': {'constant_volume' : True}" --no-tracker
</div>


In [ ]:
! janus geomopt --config geomopt_config_1.yml

Visualising this:

In [ ]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

## Constant pressure and symmetry refinement

We can also choose to optimise at a fixed pressure (in GPa), and refine the symmetry of the final structure:

In [ ]:
! janus geomopt --arch mace_mp --struct ../data/NaCl-deformed.xyz --write-traj --pressure 10 --opt-cell-fully --symmetrize --no-tracker

Visualising this:

In [ ]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

## Comparing MACE to Orb

Finally, let's compare to the structure optimised by Orb:

In [ ]:
%%writefile geomopt_config_mace.yml

struct: ../data/NaCl-deformed.xyz
arch: mace_mp
opt_cell_fully: True
minimize_kwargs:
  filter_kwargs:
    constant_volume: True
pressure: 10
file_prefix: janus_results/NaCl-mace
tracker: False

In [ ]:
! janus geomopt --config geomopt_config_mace.yml
! janus geomopt --config geomopt_config_mace.yml --arch orb --file-prefix janus_results/NaCl-orb

Visualising the final structures:

In [ ]:
from ase.io import read
from weas_widget import WeasWidget

mace_opt = read("janus_results/NaCl-mace-opt.extxyz")
orb_opt = read("janus_results/NaCl-orb-opt.extxyz")
opt_comparison = [mace_opt, orb_opt]

v=WeasWidget()
v.from_ase(opt_comparison)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v